### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Celiac_Disease/GSE106260'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
def check_gene_expression(sample_characteristics):
    # Implement the logic to check gene expression data availability
    for key, values in sample_characteristics.items():
        for value in values:
            if "expression" in value or "gene" in value:
                return True
    return False

# Expanded Sample Characteristics Dictionary for demonstration
sample_characteristics = {
    0: ['cell type: intestinal epithelial cells', 'cell type: intraepithelial lymphocytes'],
    1: ['disease state: Celiac_Disease'],
    2: ['age: 10', 'age: 12'],
    3: ['gender: female', 'gender: male']
}

# Set the gene expression flag based on availability check
is_gene_available = check_gene_expression(sample_characteristics)

# Accurate extraction of variable rows with debug logs
def get_variable_row(feature_name):
    for key, values in sample_characteristics.items():
        for value in values:
            if feature_name.lower() in value.lower():
                print(f"Identified {feature_name} in sample_characteristics with key: {key}")
                return key
    return None

# Identify rows
trait_row = get_variable_row('celiac_disease')
age_row = get_variable_row('age')
gender_row = get_variable_row('gender')

# Verify identified rows
print(f"trait_row: {trait_row}, age_row: {age_row}, gender_row: {gender_row}")

# Conversion functions
def convert_trait(value):
    value = value.split(":")[-1].strip()
    if value in ["0", "1"]:
        return int(value)
    return None

def convert_age(value):
    value = value.split(":")[-1].strip()
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    value = value.split(":")[-1].strip().lower()
    if value == 'male':
        return 1
    elif value == 'female':
        return 0
    return None

# Save cohort information
save_cohort_info('GSE106260', './preprocessed/Celiac_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Celiac_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Celiac_Disease/trait_data/GSE106260.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
else:
    print("Clinical data not available; trait_row is None.")
